In [ ]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)                                      #2 Second Timeout
result = sock.connect_ex(('127.0.0.1', 8889))
if result == 0:
    print( 'pigpiod is running' )
else:
    !sudo pkill pigpiod
    !sudo pigpiod -p 8889 ### Start PiGPIO Server / Deamon          http://abyz.me.uk/rpi/pigpio/pigpiod.html
    print( 'pigpiod was started' )

In [ ]:
import io, os, sys, ssl, time, json, socket, struct, random, datetime, threading
import serial, pigpio, picamera
from adafruit_motorkit import MotorKit
from IPython.display import clear_output

server_ip = 'yzlab3.chem.nyu.edu'
image_port = 54321
sensor_port = 54324
control_port = 54322 

server_cert_self_signed  = 'secret/server.crt' 
client_certs_self_signed = 'secret/rover.crt'
client_key_self_signed   = 'secret/rover.key'

# SSL context
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert_self_signed)
context.load_cert_chain(certfile=client_certs_self_signed, keyfile=client_key_self_signed)

#kit = MotorKit() 
pi = pigpio.pi(port=8889) ### Start PiGPIO Client
rotation_pin, elevation_pin = 17, 27  

In [ ]:
def control_link_daemon():
    l = struct.calcsize('<hf')
    while True:
        try: 
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            ss.settimeout(30)
            print('Try to connect the server') 
            ss.connect((server_ip, control_port))
            print('Succeed a connection')
            while True: 
                code, value = struct.unpack('<hf', ss.recv(l)) 
                #print(code, value)
                if code == -1:
                    pi.set_servo_pulsewidth(rotation_pin, 0) 
                    pi.set_servo_pulsewidth(elevation_pin, 0) 
                elif code == 2: pi.set_servo_pulsewidth(rotation_pin, value) 
                elif code == 3: pi.set_servo_pulsewidth(elevation_pin, value) 
                elif code == 4: kit.motor3.throttle = value
                elif code == 5: kit.motor4.throttle = value
        except (socket.timeout, struct.error) as e:
            ss.shutdown(socket.SHUT_RDWR) 
            ss.close()
            print('Failed to receive any data from server so that control channel closes and will reopen in 5 seconds', e)
            time.sleep(5)      
        except ConnectionRefusedError:
            print('Failed to connect the server and retry in 5 seconds')
            time.sleep(5)
threading.Thread(target=control_link_daemon).start()

In [ ]:
#nano = serial.Serial(port='/dev/ttyUSB0', baudrate=115200, timeout=3) # serial commu with gps chip
#while 1:    print(nano.readline())

def sensor_link_daemon():   
    nano = serial.Serial(port='/dev/ttyUSB0', baudrate=115200, timeout=3) # serial commu with gps chip
    time.sleep(3) # wait for the readiness of sensors 
    while True: 
        while True:
            try: 
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
                ss.settimeout(5)  
                print('Try to connect the server for sensor link')
                ss.connect((server_ip, sensor_port)) 
                print('Succeed a connection') 
                break
            except (ConnectionRefusedError, ConnectionResetError):
                print('Failed to connect the server and retry the connection in 5 seconds')
                time.sleep(5)
                continue
        try:
            while True:
                time.sleep(5) 
                data = nano.readline()
                ss.send(struct.pack('<H', len(data)))
                ss.send(data) 
        except socket.timeout as e: 
            ss.shutdown(socket.SHUT_RDWR)
            ss.close()
            print('[Sensor Link] Time out and closed. Reestablish in 5 seconds:', e)
            time.sleep(5)
            break
        except (ConnectionResetError, BrokenPipeError) as e:
            print('[Sensor Link] Server ended the connection. Reestablish in 5 seconds:', e)
            time.sleep(5) 
            break 
    # nano.close()
threading.Thread(target=sensor_link_daemon).start() 

In [ ]:
def image_link_daemon():
# https://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream
# https://picamera.readthedocs.io/en/release-1.13/recipes2.html#web-streaming
    camera = picamera.PiCamera(resolution=(640, 480)) 
    camera.rotation = 270
    camera.start_preview() 
    time.sleep(2)
    while True:
        while True:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
                ss.settimeout(5)  
                print('[Image Link] Try to connect the server')
                ss.connect((server_ip, img_port)) 
                print('[Image Link] Succeed a connection') 
                img_io = ss.makefile('wb')
                stream = io.BytesIO()
                break
            except (ConnectionRefusedError, ConnectionResetError):
                print('[Image Link] Failed to connect the server. Reestablish in 5 seconds')
                time.sleep(5)
                continue 
        try:
            for foo in camera.capture_continuous(stream, 'jpeg'):  
                #camera.annotate_text = f'Celsius:{sensor.Temperature.rstrip()} {dt}'  
                img_io.write(struct.pack('<L', stream.tell()))
                img_io.flush()
                stream.seek(0)
                img_io.write(stream.read()) 
                img_io.flush()
                stream.seek(0)
                stream.truncate()
                time.sleep(1)
        except socket.timeout as e: 
            ss.shutdown(socket.SHUT_RDWR)
            ss.close()
            print('[Image Link] Time out and closed. Reestablish in 5 seconds:', e)
            time.sleep(5) 
        except (ConnectionResetError, BrokenPipeError) as e:
            print('[Image Link] Server ended the connection. Reestablish in 5 seconds:', e)
            time.sleep(5)
    
threading.Thread(target=image_link_daemon).start()  